# Config

In [20]:
#editamos el path para poder accedera las funciones con la estructura actual de carpetas
from sys import path
import os
path.append(os.path.realpath('../'))

In [21]:
#importamos las funciones que hemos creado nosotros
from utils import functions as func

In [22]:
#importamos librería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

import folium
import plotly.express as px
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [23]:
# Leemos los ficheros
ruta = '../data/'

df_calendar = pd.read_csv(ruta+'daily_calendar_with_events.csv')
df_sales = pd.read_csv(ruta+'item_sales.csv')
df_prices = pd.read_csv(ruta+'item_prices.csv')

# Datasets info

## Calendar

In [24]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [25]:
df_calendar.value_counts()

date        weekday    weekday_int  d       event         
2011-02-06  Sunday     2            d_9     SuperBowl         1
2011-08-01  Monday     3            d_185   Ramadan starts    1
2016-02-07  Sunday     2            d_1836  SuperBowl         1
2016-01-01  Friday     7            d_1799  NewYear           1
2015-11-26  Thursday   6            d_1763  Thanksgiving      1
2015-06-18  Thursday   6            d_1602  Ramadan starts    1
2015-04-05  Sunday     2            d_1528  Easter            1
2015-02-01  Sunday     2            d_1465  SuperBowl         1
2015-01-01  Thursday   6            d_1434  NewYear           1
2014-11-27  Thursday   6            d_1399  Thanksgiving      1
2014-06-29  Sunday     2            d_1248  Ramadan starts    1
2014-04-20  Sunday     2            d_1178  Easter            1
2014-02-02  Sunday     2            d_1101  SuperBowl         1
2014-01-01  Wednesday  5            d_1069  NewYear           1
2013-11-28  Thursday   6            d_1035  T

In [26]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [27]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [28]:
df_calendar["yearweek"]= df_calendar['date'].apply(func.calcular_yearweek)

NameError: name 'dt' is not defined

In [ ]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')

In [ ]:
df_calendar["yearweek"].describe()

count      1913.000000
mean     201346.884997
std         151.659655
min      201104.000000
25%      201221.000000
50%      201337.000000
75%      201501.000000
max      201616.000000
Name: yearweek, dtype: float64

## Sales

In [ ]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


Pasamos las columnas dia a fila

In [ ]:
#Pasamos las columas d a filas par poder trabajar con ellas
# Cogemos las columnas.
columnas = df_sales.columns
list_id_vars = columnas[:7]   # son las columnas que necesitamos mantener.
list_value_vars = columnas[7:]

# Codigo para trasponer columnas a filas.
df_daily_sales = pd.melt(df_sales, id_vars=list_id_vars,
                   value_vars=list_value_vars ,
                   var_name='d', value_name='qty_sold')

In [ ]:
df_daily_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   qty_sold    int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


## Prices

In [ ]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [ ]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [ ]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [ ]:
df_prices=df_prices.dropna(subset=["yearweek"])
df_prices.isnull().sum()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [ ]:
# Pasamos la variable yearweek a modo fecha:
df_prices["yearweek"] = df_prices["yearweek"].astype("int64")

In [ ]:
# Para hacer merge con el df_prices creamos columna d que es item+store_code dentro de prices
df_prices["id"]= df_prices["item"]+"_"+df_prices['store_code']

KeyboardInterrupt: 

In [ ]:
df_prices.head()

,item,category,store_code,yearweek,sell_price,id
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414,ACCESORIES_1_001_NYC_1
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.9858,ACCESORIES_1_001_NYC_1
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.9858,ACCESORIES_1_001_NYC_1
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.9858,ACCESORIES_1_001_NYC_1


In [ ]:
del df_prices["item"], df_prices['store_code'], df_prices['category']

In [ ]:
df_prices["yearweek"].describe()

count    6.721786e+06
mean     2.013824e+05
std      1.450189e+02
min      2.011050e+05
25%      2.012480e+05
50%      2.014100e+05
75%      2.015150e+05
max      2.016170e+05
Name: yearweek, dtype: float64

### Imputacion nulos en sellprice

In [ ]:
#generamos las combinaciones posibles de semana y id, para luego generar las observaciones con nulos 
all_combinations = pd.MultiIndex.from_product([df_prices["id"].unique(), df_prices["yearweek"].unique()], names=['id', 'yearweek'])
df_yearweek = pd.DataFrame(index=all_combinations).reset_index()

In [ ]:
print('df_yearweek')
print('shape:')
print(df_yearweek.shape)
print('unique:')
print(df_yearweek.nunique())
print()
print('df_prices')
print('shape:')
print(df_prices.shape)
print('unique:')
print(df_prices.nunique())

df_yearweek
shape:
(8506710, 2)
unique:
id          30490
yearweek      279
dtype: int64

df_prices
shape:
(6721786, 3)
unique:
yearweek        279
sell_price     1860
id            30490
dtype: int64


In [ ]:
df_prices = pd.merge(df_yearweek, df_prices, on=['id', 'yearweek'], how='left')

In [ ]:
print('df_prices')
print('shape:')
print(df_prices.shape)
print('unique:')
print(df_prices.nunique())
print('nulos')
df_prices.isnull().sum()

df_prices
shape:
(8506710, 3)
unique:
id            30490
yearweek        279
sell_price     1860
dtype: int64
nulos


id                  0
yearweek            0
sell_price    1784924
dtype: int64

In [ ]:
df_prices.sort_values(['id', 'yearweek'], ascending = True, inplace = True)

Para ver si hacemos bien la imputacion seleccionamos algun id con nulos para comprobar

In [ ]:
rows_with_null_sell_price = df_prices[df_prices['sell_price'].isnull()]

# Seleccionar un ID que tenga al menos una semana con 'sell_price' nulo
selected_id = rows_with_null_sell_price['id'].sample(n=1).iloc[0]

# Mostrar el ID seleccionado
print(f"ID seleccionado: {selected_id}")

ID seleccionado: SUPERMARKET_1_184_PHI_2


In [ ]:
sample_ts = df_prices[df_prices['id']=='ACCESORIES_2_079_BOS_3']

In [ ]:
df_prices["sell_price"] = df_prices.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill())

# Union datasets

## Calendar sales 
Realizamos la union del df_calendar y el df_daily sales a través de la variable d.

In [ ]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
 5   yearweek     1913 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 89.8+ KB


In [ ]:
df_calendar_sales = pd.merge(df_daily_sales,df_calendar[['date','d']], on='d', how='inner')

In [ ]:
df_calendar_sales.head()

,id,item,category,department,store,store_code,region,d,qty_sold,date
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29


In [ ]:
df_calendar_sales.shape

(58327370, 10)

## Reducimos dimensionalidad

In [ ]:
df_calendar_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 10 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   qty_sold    int64 
 9   date        object
dtypes: int64(1), object(9)
memory usage: 4.3+ GB


In [ ]:
df_weekly_sales = (df_calendar_sales
                   .groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region', pd.Grouper(key='date', freq='W-Sun')], as_index=False)
                   ['qty_sold'].sum()
                   )

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:

df_weekly_sales["yearweek"]=df_weekly_sales['date'].apply(func.calcular_yearweek)

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'


- pd.Grouper es una clase en la biblioteca pandas que proporciona una interfaz flexible para realizar operaciones de agrupación en series temporales. Se utiliza comúnmente con el método groupby para agrupar datos temporales de manera conveniente y realizar operaciones de agregación en esos grupos.
- key='date': Especifica la columna por la cual se realizará la agrupación. En este caso, la columna 'date'.
- freq='W-Sun': Especifica la frecuencia con la que se agruparán los datos. En este ejemplo, se agruparán semanalmente ('W') y el doming ('sun') será el día de inicio de la semana.

In [ ]:
df_weekly_sales.shape

(8354260, 10)

In [ ]:
df_weekly_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   date        datetime64[ns]
 8   qty_sold    int64         
 9   yearweek    int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 637.4+ MB


## Union Weekly Sales con Prices

In [ ]:
df_merged_final= pd.merge(df_weekly_sales, df_prices, on=['id', "yearweek"], how='left')  

In [ ]:
df_merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   date        datetime64[ns]
 8   qty_sold    int64         
 9   yearweek    int64         
 10  sell_price  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 701.1+ MB


In [ ]:
df_merged_final.to_csv((ruta+'df_merged.csv'), index=False)